<a href="https://colab.research.google.com/github/janghyukjin/MachineLearning/blob/master/Assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd drive/My\ Drive/machineLearning


In [0]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.tag import pos_tag, untag
import matplotlib.pyplot as plt
import numpy as np
print("~")
review_data = load_files(r"movie_review")

X, y = review_data.data, review_data.target
print(len(X))
print(len(y))
documents = []

nltk.download('averaged_perceptron_tagger')

stemmer = WordNetLemmatizer()
print("~!")
for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    document = document.split()
    document = [word for word in document if len(word)>=3]
    tagged_list = pos_tag(document)

    document = ' '.join(document)
    final_doc = []
    print("!!")
    for t in tagged_list:
      if t[1] in ["NN", "NNP", "NNS", "NNPS"]:
        tmp = stemmer.lemmatize(t[0], pos="n")
        final_doc.append(tmp)
      elif t[1] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
        tmp = stemmer.lemmatize(t[0], pos="v")
        final_doc.append(tmp)
      elif t[1] in ["JJ", "JJR", "JJS"]:
        tmp = stemmer.lemmatize(t[0], pos="a")
        final_doc.append(tmp)
      elif t[1] in ["RB", "RBR", "RBS", "RP"]:
        tmp = stemmer.lemmatize(t[0], pos="r")
        final_doc.append(tmp)
      else:
        tmp = stemmer.lemmatize(t[0])
        final_doc.append(tmp)
    final_doc = ' '.join(final_doc)

    documents.append(final_doc)




In [0]:
vectorizer = CountVectorizer(max_features=1200, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
print(len(X))
print(len(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
v=np.random.randn(50,1200) 
w=np.random.randn(1,50) 
m = 1400
test_m = 600
cost = 0
test_cost = 0
accuracy = 0
test_accuracy = 0
cost_list = np.empty(0)
accuracy_list = np.empty(0)
test_cost_list = np.empty(0)
test_accuracy_list = np.empty(0)
bias2 = np.zeros((50, 1)) 
bias3 = np.zeros((1, 1))
lamda = 0.5
cnt = 0


def sigmoid(z):
  return 1/(1+np.exp(-z))

In [0]:
alpha = 0.5
count = 0
jx = np.empty(0)
final_cnt = 0
while(count<20000):
  if count%10==0:
    print(count)
  
  jx = np.append(jx, count)
  count += 1
  
  #propagation

  bias_z = np.dot(v, X_train.T)
  bias_z += bias2 
  sig_z = sigmoid(bias_z)
  bias_h = np.dot(w, sig_z)
  bias_h += bias3
  sig_h = sigmoid(bias_h)
  
  #objective Function
  cost = (-1/1400)*np.sum(y_train*np.log(sig_h) + (1-y_train)*np.log(1-sig_h)) + (lamda/(2*1400))*(np.sum(np.square(v)) + np.sum(np.square(w)))
  cost_list = np.append(cost_list,cost)
  print("cost :                 ",cost)
  #Calculate Accuracy
  accuracy = 0

  for i in range(1401):
    if sig_h[0][i]>=0.5:
      sig_h[0][i] = 1
    else:
      sig_h[0][i] = 0
    if y_train[i] == sig_h[0][i]:
      accuracy += 1
  accuracy/=1400
  accuracy*=100 
  print("training :       ",accuracy)
  accuracy_list = np.append(accuracy_list, accuracy)

  #back propagation
  delta_h = (sig_h-y_train)
  delta_z = np.dot(np.transpose(w),delta_h)*sig_z*(1-sig_z)

  delta_bias3 = np.sum(delta_h, axis=1, keepdims=True)/1000
  delta_bias2 = np.sum(delta_z, axis=1, keepdims=True)/1000

  delta_w = np.dot(delta_h,np.transpose(sig_z))
  delta_v = np.dot(delta_z,X_train)
  delta_w/=1000
  delta_v/=1000
  delta_w += (lamda/(1000))*w
  delta_v += (lamda/(1000))*v

  #update theta, bias
  v -= alpha * delta_v 
  w -= alpha * delta_w 
  bias2 -= alpha * delta_bias2 
  bias3 -= alpha * delta_bias3

  #test images propagation

  test_bias_z = np.dot(v, X_test.T)
  test_bias_z += bias2 
  test_sig_z = sigmoid(test_bias_z)
  test_bias_h = np.dot(w, test_sig_z)
  test_bias_h += bias3
  test_sig_h = sigmoid(test_bias_h)
  #test images objective function
  test_cost = (-1/600)*np.sum(y_test*np.log(test_sig_h) + (1-y_test)*np.log(1-test_sig_h)) + (lamda/(2*600))*(np.sum(np.square(v)) + np.sum(np.square(w)))
  test_cost_list = np.append(test_cost_list,test_cost)
  #test images accuracy
  test_accuracy = 0

  for i in range(601):
    if test_sig_h[0][i]>=0.5:
      test_sig_h[0][i] = 1
    else:
      test_sig_h[0][i] = 0
    if y_test[i] == test_sig_h[0][i]:
      test_accuracy += 1
  test_accuracy/=600
  test_accuracy*=100 
  print("test :", test_accuracy)
  test_accuracy_list = np.append(test_accuracy_list, test_accuracy)
  #change running rate for improving accuracy
  if test_accuracy> 77:
    alpha = 0.05
  elif test_accuracy > 65: 
    alpha = 0.01

  if test_accuracy > 82: 
    break


In [0]:
y_pred_train = []
y_pred_test = []
for s in range(1):

    test_bias_z = np.dot(v, X_test.T)
    test_bias_z += bias2 
    test_sig_z = sigmoid(test_bias_z)
    test_bias_h = np.dot(w, test_sig_z)
    test_bias_h += bias3
    test_sig_h = sigmoid(test_bias_h)
    for i in range(1401):
      if sig_h[0][i]>=0.5:
        sig_h[0][i] = 1
      else:
        sig_h[0][i] = 0
      y_pred_train.append(sig_h[0][i])

    for i in range(601):
      if test_sig_h[0][i]>=0.5:
        test_sig_h[0][i] = 1
      else:
        test_sig_h[0][i] = 0
      y_pred_test.append(test_sig_h[0][i])

# **Plot the loss curve**

# **Plot the accuracy curve**

# **Plot the quantitative results**